In [17]:
print("Ok")

Ok


In [18]:
%pwd

'c:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\research'

In [19]:
import os
os.chdir("../")

In [66]:
%pwd

'c:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot'

In [21]:
pip install jq

Note: you may need to restart the kernel to use updated packages.


In [67]:
from langchain.document_loaders import JSONLoader, PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import jq

In [68]:
def load_json_directory(data):
    loader = DirectoryLoader(
        data,
        glob="*.json",
        loader_cls=JSONLoader,
        loader_kwargs={
            "jq_schema": ".[]",
            "content_key": "description",
            "text_content": True
        }
       
    )
    documents = loader.load()
    return documents

In [69]:
extracted_data=load_json_directory(data='PreprocessedData')

In [70]:
extracted_data

[Document(metadata={'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json', 'seq_num': 1}, page_content='live horses, asses, mules and hinnies'),
 Document(metadata={'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json', 'seq_num': 2}, page_content='live horses, asses, mules and hinnies, horses, purebred breeding animals'),
 Document(metadata={'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json', 'seq_num': 3}, page_content='live horses, asses, mules and hinnies, horses, purebred breeding animals, males'),
 Document(metadata={'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json', 'seq_num': 4}, page_content='live horses, asses, mules and hinnies, horses, purebred breeding animals, female

In [71]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [72]:

text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 26662


In [73]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

In [74]:
# Download the embeddings from OpenAI
def download_openai_embeddings():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")  
    return embeddings

In [75]:
# #Download the Embeddings from Hugging Face
# def download_hugging_face_embeddings():
#     embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#     return embeddings

In [76]:
# embeddings = download_hugging_face_embeddings()
embeddings = download_openai_embeddings()

In [77]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))
# query_result = embeddings.embed_query("Hello world")
# print("Length", len(query_result))

Length 3072


In [78]:
from dotenv import load_dotenv
load_dotenv()

True

In [79]:

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [82]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "raterbot"


pc.create_index(
    name=index_name,
    dimension=3072, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [83]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [84]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [85]:

docsearch

In [86]:

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [87]:

retrieved_docs = retriever.invoke("Stainless Steel")

In [88]:

retrieved_docs

[Document(id='61c84169-ede6-45c0-a82e-2f470fb66dac', metadata={'seq_num': 14344.0, 'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json'}, page_content='wire of stainless steel'),
 Document(id='9e03634e-2e86-447e-b74f-24668b4f5e96', metadata={'seq_num': 14809.0, 'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json'}, page_content='doors, of stainless steel'),
 Document(id='2774a44e-7ec0-4c8c-873c-68753bc5222a', metadata={'seq_num': 14345.0, 'source': 'C:\\Users\\t000022u\\OneDrive - Trench Group\\Desktop\\Project1\\Rater-Bot\\PreprocessedData\\final_rag_hts_data.json'}, page_content='wire of stainless steel, round wire')]

In [89]:

# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.1, max_tokens=500)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.1,
    max_tokens=500
)

In [90]:

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
    
system_prompt = ("""
You are an AI customs classification assistant. Your task is to determine the most accurate Harmonized System Tariff (HST) code for a given product description. Utilize the provided reference materials, including tariff schedules and classification guidelines, to support your decision. Ensure that your response includes:

- The recommended HST code.
- A brief explanation of the classification rationale.
- Any relevant notes or considerations from the reference materials.

Only respond if sufficient relevant context is provided. Do not guess or assume details not found in the context. Do not invent product features.

Context: {context}
Question: {input}

Example 1:
Input: Stainless steel machine screw, 6.35 mm shank diameter, hex washer head 9.53 mm.
Context: HTS 7318.15.80.30 – Other screws and bolts, whether or not with their nuts or washers: Other: Having shanks or threads with a diameter of 6 mm or more: Of stainless steel.
Output:
Recommended HST code: 7318.15.80.30
Classification rationale: The product is a stainless steel screw with a shank diameter exceeding 6 mm, fitting the specified category.
Notes: Ensure the screw is not self-tapping or part of a special set, as those may fall under different codes.

Example 2:
Input: Self-tapping screw anchor, stainless steel, 6.35 mm shank diameter, hex washer head 9.53 mm.
Context: HTS 7318.14.50.20 – Self-tapping screws, of stainless steel, having shanks or threads with a diameter of 6 mm or more.
Output:
Recommended HST code: 7318.14.50.20
Classification rationale: The product is a self-tapping screw made of stainless steel with a diameter exceeding 6 mm, aligning with the specified HTS code.
Notes: Confirm that the product is indeed self-tapping; otherwise, a different classification may apply.
""")


In [91]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [92]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [93]:
# Invoke the chain with user input
user_input = "Determine the appropriate Harmonized System Tariff (HST) code for a stainless steel screw with a shank diameter of 6 mm or more."
response = rag_chain.invoke({"input": user_input})
print(response["answer"])

Recommended HST code: 7318.15.80.30

Classification rationale: The product described is a stainless steel screw with a shank diameter of 6 mm or more. Based on the provided context, it fits within the category specified by HTS 7318.15.80.30, which covers "Other screws and bolts, whether or not with their nuts or washers: Other: Having shanks or threads with a diameter of 6 mm or more: Of stainless steel."

Notes: It is important to confirm that the screw is not a self-tapping screw or part of a special set, as those characteristics could necessitate a different classification under the Harmonized System Tariff codes. Additionally, ensure that the screw does not have any unique features that might place it under a different classification.


In [94]:
user_input = "Self-tapping screw, stainless steel, 6.35 mm shank diameter."
response = rag_chain.invoke({"input": user_input})
print(response["answer"])

Recommended HST code: 7318.14.50.20

Classification rationale: The product described is a self-tapping screw made of stainless steel with a shank diameter of 6.35 mm. This specification fits within the category of self-tapping screws of stainless steel having shanks or threads with a diameter of 6 mm or more, as outlined in the provided HTS code.

Notes: It is important to confirm that the screw is indeed self-tapping as this classification specifically applies to self-tapping screws. If the screw has different features (e.g., not self-tapping), it may require a different classification.


In [95]:
user_input = "Socket screw, stainless steel, 7 mm thread diameter."
response = rag_chain.invoke({"input": user_input})
print(response["answer"])

Recommended HST code: 7318.15.80.30

Classification rationale: The product described is a socket screw made of stainless steel with a thread diameter of 7 mm. This fits within the category of "Other screws and bolts, whether or not with their nuts or washers: Other: Having shanks or threads with a diameter of 6 mm or more: Of stainless steel," as per the provided context and classification guidelines.

Notes: It is important to verify that the screw is not designed for a specific application that might classify it differently (e.g., automotive or aerospace specific), and that it does not include additional features that could alter its classification (such as being part of a set or assembly).


In [96]:
user_input = "Machine screw, stainless steel, 10 mm length, 3.5 mm diameter."
response = rag_chain.invoke({"input": user_input})
print(response["answer"])

Recommended HST code: 7318.15.80.10

Classification rationale: The product described is a machine screw made of stainless steel, with a length of 10 mm and a diameter of 3.5 mm. According to the Harmonized Tariff Schedule, machine screws of stainless steel that have a diameter of 3.2 mm or more but less than 6 mm are classified under 7318.15.80.10. This code specifically covers screws, bolts, nuts, and similar articles of iron or steel, threaded articles, other screws and bolts, whether or not with their nuts or washers, other, having shanks or threads with a diameter of less than 6 mm, of stainless steel.

Notes: It is important to confirm that the screw is not designed for any specific applications that might categorize it differently (e.g., as a cap screw or a self-tapping screw). The classification is based on the general description of a machine screw with the specified dimensions and material.


In [57]:
user_input = "Structural bolt, stainless steel, 8 mm diameter."
response = rag_chain.invoke({"input": user_input})
print(response["answer"])


Recommended HST code: 7318.15.80.30
Classification rationale: The product is a structural bolt made of stainless steel with a diameter of 8 mm, fitting the specified category.
Notes: Ensure the bolt is not part of a special set, as those may fall under different codes.


In [97]:
user_input = "Self-tapping screw anchor, stainless steel, 6.35 mm shank diameter, internally threaded hex washer head."
response = rag_chain.invoke({"input": user_input})
print(response["answer"])

Recommended HST code: 7318.14.50.20

Classification rationale: The product described is a self-tapping screw anchor made of stainless steel with a shank diameter of 6.35 mm. This fits the criteria for HTS 7318.14.50.20, which specifically categorizes self-tapping screws of stainless steel having shanks or threads with a diameter of 6 mm or more.

Notes: It is important to verify that the screw is self-tapping as this classification is specific to self-tapping screws. If the screw is not self-tapping, it would require a different classification code.
